In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from langchain.llms import HuggingFacePipeline
from transformers import pipeline

class LangChainHandler:
    def __init__(self, model="gpt-4", api_key=None, local_model_name=None):
        """
        Initialize the LangChainHandler.
        
        Args:
            model (str): The model to use ("gpt-4" for OpenAI, or local Hugging Face model).
            api_key (str): OpenAI API key (if using OpenAI model).
            local_model_name (str): Local model name for Hugging Face (if using local model).
        """
        if api_key:
            self.llm = ChatOpenAI(model=model, openai_api_key=api_key)
        elif local_model_name:
            local_pipeline = pipeline("text-generation", model=local_model_name)
            self.llm = HuggingFacePipeline(pipeline=local_pipeline)
        else:
            raise ValueError("Either `api_key` or `local_model_name` must be provided.")

    def generate_response(self, prompt_template: str, variables: dict) -> str:
        """
        Generate a response using LangChain.
        
        Args:
            prompt_template (str): The template for the prompt.
            variables (dict): The variables to fill in the prompt.
            
        Returns:
            str: The generated response.
        """
        prompt = PromptTemplate(input_variables=list(variables.keys()), template=prompt_template)
        chain = LLMChain(llm=self.llm, prompt=prompt)
        response = chain.run(variables)
        return response

In [8]:
# OpenAI model
openai_handler = LangChainHandler(model="gpt-4o-mini",
                                   api_key="")


# Local Hugging Face model
# local_handler = LangChainHandler(local_model_name="gpt2")

# Define a prompt template
prompt_template = """
You are {player_name}, an AI player in a Werewolf-themed game. The current state of the game is as follows:

- **Current Phase**: {current_phase} Phase
- **Current Day**: Day {current_day}
- **Alive Players**: {alive_players}
- **Dead Players**: {dead_players}
- **Game Over**: {game_over}
- **Winner**: {winner}

Here is the **Chat History** of the conversation so far:
{formatted_chat_history}

Your Role: {player_role}

### Rules for Your Response:
1. If you want to contribute to the conversation, compose a message and send it.
2. If you don't want to say anything right now, wait silently.

### Goals Based on Your Role:
- **Werewolf**: Be sneaky, avoid suspicion, and manipulate the villagers.
- **Villager**: Be cooperative, identify the werewolves, and convince others to trust you.

Compose a response that aligns with your role's goals or wait silently for now.
"""

state_summary = {
    "current_phase": "Debate",
    "current_day": 1,
    "alive_players": "Dudu, Ivan, Michael, John, Zack",
    "dead_players": '',
    "chat_history": [{'day': 1, 'phase': 'Day', 'sender': 'Dudu', 'text': 'Hi !'}, 
                      {'day': 1, 'phase': 'Day', 'sender': 'Ivan', 'text': 'Hello everyone'}, 
                      {'day': 1, 'phase': 'Day', 'sender': 'John', 'text': 'I m sure Zack is a Werewolf. We should vote for him'}],
    "game_over": False,
    "winner": None,
    "player_name": "Zach",
    "player_role": "Villager"
}

def format_chat_history(chat_history):
    return "\n".join([f"Day {entry['day']} ({entry['phase']} Phase) - {entry['sender']}: {entry['text']}" for entry in chat_history])

state_summary['formatted_chat_history'] = format_chat_history(state_summary['chat_history'])

state_summary

# # Generate a response using OpenAI
response = openai_handler.generate_response(prompt_template, state_summary)
print("OpenAI Response:", response)


OpenAI Response: Day 1 (Day Phase) - Zack: Hey everyone! I think it's important we don't jump to conclusions too quickly. John, do you have any reasons for suspecting me? It would be helpful to hear everyone's thoughts so we can figure out who the werewolves might be together. Let’s share our observations!
